# Manual tracing for OpenAI LLM calls using Maxim SDK


## Initialize SDK


In [ ]:
import os
from maxim import Maxim
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Retrieve API keys from environment variables
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# Set up Maxim logger configuration
logger = Maxim().logger()

## Create a trace and add it to the session

A trace represents one round trip between user <-> agent. More info [here]()

In [ ]:
from openai import OpenAI
from maxim.logger.openai import MaximOpenAIClient

client = MaximOpenAIClient(client=OpenAI(api_key=OPENAI_API_KEY),logger=logger)

messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Write a haiku about recursion in programming."},
]

# Create a chat completion request
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,                    
)
# Extract response text and usage
response_text = response.choices[0].message.content
print(response_text)
    

# With completions parse


In [ ]:
from pydantic import BaseModel
from typing import List

# Define the response structure using Pydantic
class HaikuAnalysis(BaseModel):
    haiku: str
    syllable_count: List[int]  # syllables per line
    theme: str
    mood: str
    follows_5_7_5_pattern: bool

# Use chat.completions.parse for structured output
messages = [
    {"role": "system", "content": "You are a poetry expert. Analyze haikus and return structured data."},
    {"role": "user", "content": "Write a haiku about bifrost and analyze it."},
]

# Create a structured completion request using parse
response = client.chat.completions.parse(
    model="gpt-4o-mini",
    messages=messages,
    response_format=HaikuAnalysis,
)

# Extract the parsed response
parsed_response = response.choices[0].message.parsed
print("Parsed Haiku Analysis:")
print(f"Haiku: {parsed_response.haiku}")
print(f"Syllable Count: {parsed_response.syllable_count}")
print(f"Theme: {parsed_response.theme}")
print(f"Mood: {parsed_response.mood}")
print(f"Follows 5-7-5 Pattern: {parsed_response.follows_5_7_5_pattern}")

# You can also access the raw content
print("\nRaw JSON:")
print(response.choices[0].message.content)


In [ ]:
## Another example: Recipe extraction with structured output


In [ ]:
from typing import Optional
from enum import Enum

class DifficultyLevel(str, Enum):
    EASY = "easy"
    MEDIUM = "medium"
    HARD = "hard"

class Ingredient(BaseModel):
    name: str
    amount: str
    unit: Optional[str] = None

class Recipe(BaseModel):
    name: str
    description: str
    prep_time_minutes: int
    cook_time_minutes: int
    servings: int
    difficulty: DifficultyLevel
    ingredients: List[Ingredient]
    instructions: List[str]
    tags: List[str]

# Request a recipe with structured output
recipe_messages = [
    {"role": "system", "content": "You are a professional chef. Create detailed recipes with precise measurements and clear instructions."},
    {"role": "user", "content": "Create a recipe for chocolate chip cookies that's suitable for beginners."},
]

recipe_response = client.chat.completions.parse(
    model="gpt-4o-mini",
    messages=recipe_messages,
    response_format=Recipe,
)

# Extract and display the structured recipe
recipe = recipe_response.choices[0].message.parsed

print(f"Recipe: {recipe.name}")
print(f"Description: {recipe.description}")
print(f"Prep Time: {recipe.prep_time_minutes} minutes")
print(f"Cook Time: {recipe.cook_time_minutes} minutes")
print(f"Servings: {recipe.servings}")
print(f"Difficulty: {recipe.difficulty.value}")

print("\nIngredients:")
for ingredient in recipe.ingredients:
    unit_text = f" {ingredient.unit}" if ingredient.unit else ""
    print(f"- {ingredient.amount}{unit_text} {ingredient.name}")

print("\nInstructions:")
for i, instruction in enumerate(recipe.instructions, 1):
    print(f"{i}. {instruction}")

print(f"\nTags: {', '.join(recipe.tags)}")


In [ ]:
## Example with error handling and validation


In [ ]:
from pydantic import Field, validator
from datetime import datetime

class EventDetails(BaseModel):
    title: str = Field(..., min_length=1, max_length=100)
    date: str = Field(..., description="Date in YYYY-MM-DD format")
    time: str = Field(..., description="Time in HH:MM format (24-hour)")
    location: str
    attendees: int = Field(..., ge=1, le=1000)  # Between 1 and 1000 attendees
    category: str = Field(..., description="Event category")
    is_virtual: bool = False
    
    @validator('date')
    def validate_date_format(cls, v):
        try:
            datetime.strptime(v, '%Y-%m-%d')
            return v
        except ValueError:
            raise ValueError('Date must be in YYYY-MM-DD format')
    
    @validator('time')
    def validate_time_format(cls, v):
        try:
            datetime.strptime(v, '%H:%M')
            return v
        except ValueError:
            raise ValueError('Time must be in HH:MM format (24-hour)')

# Example with error handling
try:
    event_messages = [
        {"role": "system", "content": "You are an event planning assistant. Create event details with proper formatting."},
        {"role": "user", "content": "Plan a tech conference for next month with about 200 attendees in San Francisco."},
    ]
    
    event_response = client.chat.completions.parse(
        model="gpt-4o-mini",
        messages=event_messages,
        response_format=EventDetails,
    )
    
    # Check if parsing was successful
    if event_response.choices[0].message.parsed:
        event = event_response.choices[0].message.parsed
        print("✅ Successfully parsed event details:")
        print(f"Title: {event.title}")
        print(f"Date: {event.date}")
        print(f"Time: {event.time}")
        print(f"Location: {event.location}")
        print(f"Attendees: {event.attendees}")
        print(f"Category: {event.category}")
        print(f"Virtual: {event.is_virtual}")
    else:
        print("❌ Failed to parse the response")
        print("Raw content:", event_response.choices[0].message.content)
        
except Exception as e:
    print(f"Error occurred: {e}")
    print("This might happen if the model fails to generate valid structured output")
